In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load data

In [ ]:
playersdf = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/players.csv')
playsdf = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/plays.csv')
scoutdf = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/PFFScoutingData.csv')
gamesdf = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/games.csv')
t2018df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2018.csv')
#t2019df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2019.csv')
#t2020df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2020.csv')

# Identify Successful Special Teams Plays
* For Field Goals = Score a field goal
* For Punts = Pin the opponent within their own 10 yard line
* For Returns = Have a 40+ yard return

In [ ]:
plays = playsdf.fillna(0)
successfulKick = plays.loc[(plays['specialTeamsPlayType'] == 'Field Goal') & (plays['specialTeamsResult'] == 'Kick Attempt Good')]
successfulKickBlock = plays.loc[(plays['specialTeamsPlayType'] == 'Field Goal') & (plays['specialTeamsResult'] == 'Blocked Kick Attempt')]
successfulPunts = plays.loc[(plays['specialTeamsPlayType'] == 'Punt') & (plays['possessionTeam'] == plays['yardlineSide']) & (plays['yardlineNumber'] + plays['kickLength'] == 99)]
successfulReturn = plays.loc[(plays['playDescription'].str.contains('TOUCHDOWN')) &(plays['specialTeamsResult'].str.contains('Non-Special Teams Result') == 0)&(plays['kickReturnYardage'] >= 1)]

# Mark Successful Plays in the Dataset

In [ ]:
successplays = pd.concat([successfulKick,successfulKickBlock,successfulPunts,successfulReturn])
plays['Success'] = [1 if x in successplays['playId'] else 0 for x in plays['playId']]
display(plays)

# Encode Other Data

In [ ]:
playsencoded = pd.get_dummies(plays, columns=['possessionTeam','specialTeamsPlayType','specialTeamsResult','down','quarter'])
display(playsencoded)


# Find Correlation Between Successful Plays and Other Data


In [ ]:
successCorr = playsencoded.corrwith(playsencoded['Success'])

**

In [ ]:
display(successCorr.sort_values().tail(30))